In [119]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pytube import YouTube
from pymongo import MongoClient
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time
import urllib.request
import datetime
import copy

In [120]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['baidu_crawling']
collection = db['OneAsia_baidu_video_crawling']
data_list = list(collection.find({"URL": "http://www.baidu.com/link?url=5fmDTIRACq7Iklhsz9tMvu8GN4g_6DCoj-1RxLQxbDdeLcv8knpMDT-z5VK0bg8gfw3kFD8s1w9D212J8QGYfq"}, {"_id": 1, "URL": 1}))

In [122]:
data_list1 = copy.deepcopy(data_list)

In [123]:
driver = webdriver.Chrome()

In [124]:
for data in tqdm(data_list, desc="Processing Links", unit="link"):
    url = data["URL"]
    driver.get(url)
    
    try:
        # 좋아요 수 요소 찾기
        like_count_element = driver.find_element(By.CSS_SELECTOR, ".video-like-info")
        
        # 텍스트 가져오기 (좋아요 수)
        like_count_str = like_count_element.text
        
        # 좋아요 수를 숫자형으로 변환, 변환 실패 시 0으로 처리
        try:
            like_count = int(like_count_str)
        except ValueError:
            like_count = 0
        
        # MongoDB에 좋아요 필드 추가
        collection.update_one({"_id": data["_id"]}, {"$set": {"좋아요": like_count}})
    except NoSuchElementException:
        # 만약 좋아요 수를 찾을 수 없는 경우에는 0으로 처리
        collection.update_one({"_id": data["_id"]}, {"$set": {"좋아요": 0}})
    
    # 1초 대기
    time.sleep(1)

Processing Links: 100%|██████████| 1/1 [00:03<00:00,  3.24s/link]


In [125]:
# MongoDB 연결 종료
client.close()

# 드라이버 종료
driver.quit()